In [2]:
!pip install fasttext

     |████████████████████████████████| 68 kB 818 kB/s eta 0:00:01
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=4745091 sha256=5c69a1ac626a919c3a9f34a6e76841cb0debe65cf3a18a6ff7e3b9fe1bfdf3aa
  Stored in directory: /home/cherepaha/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [28]:
from collections import Counter
import json
import pandas as pd
from sklearn import cluster
import gensim.downloader as api
from pymystem3 import Mystem
from tqdm import tqdm
import fasttext
import matplotlib.pyplot as plt

In [22]:
# rns_to_upos = {}
# with open('ru-rnc.map.txt') as fin:
#     for line in fin.read().split('\n'):
#         line = line.strip()
#         if line:
#             rns, upos = line.strip().split()
#             rns_to_upos[rns] = upos

# def tag(word):
#     m = Mystem()
#     processed = m.analyze(word)[0]
#     lemma = processed["analysis"][0]["lex"].lower().strip()
#     pos = processed["analysis"][0]["gr"].split(',')[0]
#     pos = pos.split('=')[0].strip()
#     upos = rns_to_upos[pos]
#     tagged = lemma+'_'+upos
#     return tagged

# model = api.load("word2vec-ruscorpora-300")
# # model.most_similar("glass")

In [4]:
!mv /home/cherepaha/Downloads/cc.ru.300.bin.gz ./

In [6]:
!gzip -d cc.ru.300.bin.gz

In [7]:
model = fasttext.load_model('cc.ru.300.bin')

In [76]:
rss = pd.read_json('rss.json')
vk = pd.read_json('vk.json')
tg = pd.read_json('tg.json')

In [77]:
rss = rss[rss.domain != 'aktualno.ru']

In [79]:
rss.drop_duplicates('title', inplace=True)
rss.drop_duplicates('source_url', inplace=True)

In [80]:
tg.source_name.unique()

array(['primamedia.ru', 'www.vedomosti.ru', 'babr24.com', 'ria.ru',
       'tverlife.ru', 'www.liga.net', 'vm.ru', 'www.sb.by',
       'www.kommersant.ru', 'kurjer.info', 'life.ru', 'www.kt.kz',
       'www.gazeta.ru', 'deita.ru', 'www.trend.az', 'www.livekuban.ru',
       'tass.ru', 'www.mn.ru', 'www.1rnd.ru', 'www.pressball.by',
       'delo.ua', 'tvzvezda.ru', 'liter.kz', 'www.vesti.ru'], dtype=object)

In [81]:
title = rss.title.drop_duplicates()

In [82]:
title_emb = title.apply(lambda x: model[x])

In [83]:
title_emb = pd.DataFrame(title_emb.to_list(), )

In [118]:
title_clusters = cluster.DBSCAN(eps=0.12, min_samples=2).fit_predict(title_emb)

In [119]:
rss.loc[:, 'cluster'] = title_clusters

In [120]:
rss.cluster.unique()

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115,
       116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
       142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154,
       155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
       168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 17

In [126]:
rss[rss.cluster == 1].title.values

array(['Разведка США ожидает "нападения России на Украину" в 2022 году',
       'AP: разведка США ожидает наступления России на Украину в начале 2022 года'],
      dtype=object)